# Numba 0.54.0 Release Demo

This notebook contains a demonstration of new features present in the 0.54.0 release of Numba. Whilst release notes are produced as part of the [CHANGE_LOG](https://github.com/numba/numba/blob/release0.54/CHANGE_LOG), there's nothing like seeing code in action!

This release contains a lot of things including new features, updated library support and large scale internal changes/enhancements to permit extending Numba to new targets. In this notebook, the new CPU target features are demonstrated. The [CUDA target](https://numba.pydata.org/numba-doc/latest/cuda/index.html) also gained a lot of new features in 0.54.0 and [@gmarkall](https://github.com/gmarkall) has created a [demo notebook](https://mybinder.org/v2/gh/numba/numba-examples/master?filepath=notebooks%2FNumba_054_CUDA_Release_Demo.ipynb) especially for these!

Key internal/support changes in this release are:

- NumPy support is restricted to (1.17 <= x < 1.21).
- Python 3.7 is the new minimum version of Python accepted.
- Intel TBB 2021 is now supported and the new minimum version of TBB.
- The LLVM backend has been upgraded to version 11.
- Numba has directly vendored cloudpickle into its code base for use with caching and object mode.

Intel also kindly sponsored research and development that lead to three new features:

- Target extension support.
- Custom NumPy array subclass support.
- Context managed retargeting of the dispatcher.

## Python support enhancements

In [ ]:
from numba import njit
import numba
assert numba.version_info.short >= (0, 54)
from numba.core import types
import numpy as np

Numba 0.54 gains some basic `f-string` support ([@ehsantn](https://github.com/ehsantn)) and support for dictionary comprehensions ([@stuartarchibald](https://github.com/stuartarchibald)).

In [ ]:
@njit
def demo_fstring_dict_comp():
    return {x: f'string_{x * 10}' for x in range(5)}

demo_fstring_dict_comp()

It also gains support for the ``sum`` builtin ([@stuartarchibald](https://github.com/stuartarchibald))

In [ ]:
@njit
def demo_sum():
    return sum([1, 2, 3]), sum((4, 5, 6)), sum((1j, 3, np.float64(7)))
    
demo_sum()

and the ability to `hash()` `IntEnum`s ([@HPLegion](https://github.com/HPLegion))

In [ ]:
from enum import IntEnum

class Scale(IntEnum):
    C4 = 261
    D4 = 293
    E4 = 329
    F4 = 349
    G4 = 391
    A4 = 440

@njit
def demo_intenum_hash():
    return hash(Scale.D4), {Scale.C4: 'Middle C'}[Scale.C4]

demo_intenum_hash()

## NumPy support enhancements

Numba 0.54 contains support for some more NumPy functions:

* ``np.clip`` ([@arvoelke](https://github.com/arvoelke), [@gmarkall](https://github.com/gmarkall))
* ``np.rot90`` ([@braniii](https://github.com/braniii))
* ``np.swapaxes`` ([@braniii](https://github.com/braniii))
* ``np.random.dirichlet`` ([@rishi-kulkarni](https://github.com/rishi-kulkarni))
* ``np.isreal`` ([@guilhermeleobas](https://github.com/guilhermeleobas))
* ``np.iscomplex`` ([@guilhermeleobas](https://github.com/guilhermeleobas))
* ``np.isrealobj`` ([@guilhermeleobas](https://github.com/guilhermeleobas))
* ``np.iscomplexobj`` ([@guilhermeleobas](https://github.com/guilhermeleobas))
* ``np.isscalar`` ([@guilhermeleobas](https://github.com/guilhermeleobas))
* ``np.isneginf`` ([@guilhermeleobas](https://github.com/guilhermeleobas))
* ``np.isposinf`` ([@guilhermeleobas](https://github.com/guilhermeleobas))

and also enhances the support for:

* ``np.argmax()`` now supports the `axis` kwarg ([@itamarst](https://github.com/itamarst))
* ``__setitem__`` using ``0d`` arrays ([@guilhermeleobas](https://github.com/guilhermeleobas))

In [ ]:
from numba import literal_unroll

@njit
def demo_numpy_new_features():
    x = np.linspace(0, 10, 20)
    print("np.clip\n", np.clip(x, 2., 5.))
    print("np.rot90\n", np.rot90(np.arange(4).reshape((2, 2))))
    print("np.swapaxes\n", np.swapaxes(np.arange(8).reshape((2, 2, 2)), 0, 2))

    np.random.seed(0)
    print("np.random.dirichlet\n", np.random.dirichlet((2, 3, 4), 2))

    for x in literal_unroll((1, 1j, np.array([1, 2j, 3]))):
        print("x is ", x)
        print("  np.iscomplexobj(x)", np.iscomplexobj(x))
        print("  np.isreal(x)", np.isreal(x))
        print("  np.isrealobj(x)", np.isrealobj(x))
        print("  np.isscalar(x)", np.isscalar(x))
    for x in literal_unroll((1, np.inf, -np.inf, np.nan)):
        print("x is ", x)
        print("  np.isneginf(x)", np.isneginf(x))
        print("  np.isposinf(x)", np.isposinf(x))
        print("  np.iscomplex(x)", np.iscomplex(x))
        
demo_numpy_new_features()

Enhancements:

In [ ]:
@njit
def demo_numpy_enhancements():
    # Axis support in np.argmax
    x = np.arange(12).reshape((3, 2, 2))
    x[:, 1, :] = 100
    print("np.argmax(x, axis=2)\n", np.argmax(x, axis=2))
    
    # Setitem using 0d arrays
    idx = np.array(1)
    x = np.arange(10)
    x[idx] = 100
    print("Setitem of x[idx]\n", x, ", idx =", idx, ", idx.ndim =", idx.ndim)
    
demo_numpy_enhancements()

----

## See Also

- [0.54 CUDA Demo](https://mybinder.org/v2/gh/numba/numba-examples/master?filepath=notebooks%2FNumba_054_CUDA_Release_Demo.ipynb)
- [0.54 TODO involved features demo, e.g. array subclass](TODO)